In [13]:
!pip install pandas

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)'))) - skipping


In [ ]:
import pandas as pd

# file with default values
mapping_df = pd.read_excel("Ingestion Mapping.xlsx", sheet_name = 'Default Mapping 4 All')

#file to be converted/normalized/validated
database_df = pd.read_excel("Reconn SLI Field Inspection 1016-1031 (1).xlsx")

#conversion for case-insensitive
mapping_df['default values'] = mapping_df['default values'].astype(str).str.lower()

print("Database column names:", database_df.columns)

# read through every row(data) in the database_df
for index, row in database_df.iterrows():
    for mapping_index, mapping_row in mapping_df.iterrows():
        column_name = mapping_row['column name']
        try:
            value_from_db = str(row[column_name])
        except KeyError:
            print(f"KeyError: Column '{column_name}' not found in the database Dataframe.")
            continue
            
        value_from_db = str(row[mapping_row['column name']])
        
        #get matching for each column
        #mapping_row = database[(mapping_df['column name']==column)& (mapping_df['normalized name']==row['normalized name'])]

        #check if values in database matches values in mapping_df
        if  value_from_db.lower() in mapping_row['default values']:
            if 'priority' in mapping_row and not pd.isna(mapping_row['priority']):
                highest_priority = mapping_df.loc[mapping_df['column name'] == mapping_row['column name'], 'priority'].min()   
                default_value = mapping_df.loc[(mapping_df['column name']==mapping_df['column name'])&(mapping_df['priority']==highest_priority),'default values'].iloc[0]                                 
            #fetches  the default value with highest prio
            else:
                default_value = mapping_row['default values']

            database_df.at[index,mapping_row['column name']] = default_value

#save to new file
database_df.to_excel('updated_database_recon.xlsx', index = False)

In [22]:
print(mapping_df.columns)
print(database_df.columns)

Index(['column name', 'normalized name', 'default values', 'priority'], dtype='object')
Index(['_record_id', '_status', '_title', '_version', '_server_created_at',
       '_server_updated_at', '_assigned_to', '_updated_by', '_geometry',
       '_latitude',
       ...
       'gas_odor_present', 'cgi_reading_gas', 'leak_ticket_', 'leaking_items',
       'leak_related_to_corrosion', 'remarks5', 'concurrent_meter_inspections',
       'poe_photos', 'meter_inspection_photos',
       'highest_level_of_corrosion_photo'],
      dtype='object', length=126)
